In [5]:
import pandas as pd
import requests
from io import StringIO

# 엑셀 파일 경로
file_path = r'C:\Users\rkddk\OneDrive\문서\카카오톡 받은 파일\perdata.xlsx'

# 엑셀 파일 읽기 및 종목코드 추출
try:
    df = pd.read_excel(file_path)

    # B열의 종목코드 추출 및 문자열 형식으로 변환
    df['종목코드'] = df['종목코드'].astype(str)
    
    # 종목코드가 6글자가 아닌 경우 앞에 0을 채워 6글자로 만듦
    df['종목코드'] = df['종목코드'].apply(lambda x: x.zfill(6))
    
    ticker_list = df['종목코드'].tolist()

    print("종목코드 리스트:")
    print(ticker_list)
except Exception as e:
    print(f"Error reading the Excel file: {e}")

def get_value_indices(stock_code):
    url = f'https://comp.fnguide.com/SVO2/asp/SVD_Invest.asp?pGB=1&gicode=A{stock_code}&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701'
    response = requests.get(url)
    if response.status_code == 200:
        tables = pd.read_html(StringIO(response.text))
        invest_table = tables[3]
        invest_table = invest_table.set_index(invest_table.columns[0])
        try:
            value_indices = invest_table.loc[['PER계산에 참여한 계정 펼치기', 'PCR계산에 참여한 계정 펼치기', 'PSR계산에 참여한 계정 펼치기', 'PBR계산에 참여한 계정 펼치기']]
            value_indices.index = ['PER', 'PCR', 'PSR', 'PBR']
            return value_indices.T
        except KeyError:
            return None
    else:
        return None

def make_fr_dataframe(firm_code):
    fr_url = 'https://comp.fnguide.com/SVO2/asp/SVD_FinanceRatio.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=104&stkGb=701&gicode=A' + firm_code
    fr_page = requests.get(fr_url)
    if fr_page.status_code == 200:
        fr_tables = pd.read_html(fr_page.text)
        temp_df = fr_tables[0]
        temp_df = temp_df.set_index(temp_df.columns[0])
        try:
            temp_df = temp_df.loc[['영업이익률계산에 참여한 계정 펼치기', 'ROA계산에 참여한 계정 펼치기']]
            temp_df.index = ['영업이익률', 'ROA']
            return temp_df.T
        except KeyError:
            return None
    else:
        return None

def make_fs_dataframe(firm_code):
    fs_url = f'https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701&gicode=A{firm_code}'
    fs_page = requests.get(fs_url)
    if fs_page.status_code == 200:
        fs_tables = pd.read_html(fs_page.text)
        try:
            temp_df = fs_tables[0]
            temp_df = temp_df.set_index(temp_df.columns[0])
            temp_df = temp_df[temp_df.columns[:4]]
            temp_df = temp_df.loc[['매출액', '영업이익', '당기순이익']]

            temp_df3 = fs_tables[4]
            temp_df3 = temp_df3.set_index(temp_df3.columns[0])
            temp_df3 = temp_df3.loc[['영업활동으로인한현금흐름']]

            fs_df = pd.concat([temp_df, temp_df3])
            return fs_df.T
        except (KeyError, IndexError):
            return None
    else:
        return None

# 결과를 저장할 리스트
results = []

for stock_code in ticker_list:
    try:
        value_indices = get_value_indices(stock_code)
        if value_indices is not None:
            try:
                # 2022/12 데이터 추출
                per = value_indices.at['2022/12', 'PER']
                pcr = value_indices.at['2022/12', 'PCR']
                psr = value_indices.at['2022/12', 'PSR']
                pbr = value_indices.at['2022/12', 'PBR']
                
                # 재무 비율 데이터 추출
                fr_data = make_fr_dataframe(stock_code)
                if fr_data is not None:
                    operating_margin = fr_data.at['2022/12', '영업이익률']
                    roa = fr_data.at['2022/12', 'ROA']
                else:
                    operating_margin = roa = None

                # 재무 상태 데이터 추출
                fs_data = make_fs_dataframe(stock_code)
                if fs_data is not None:
                    revenue = fs_data.at['2022/12', '매출액']
                    operating_income = fs_data.at['2022/12', '영업이익']
                    net_income = fs_data.at['2022/12', '당기순이익']
                    cash_flow_from_operations = fs_data.at['2022/12', '영업활동으로인한현금흐름']
                else:
                    revenue = operating_income = net_income = cash_flow_from_operations = None

                # 결과 저장
                results.append({
                    '종목코드': stock_code,
                    '날짜': '2022/12',
                    'PER': per,
                    'PCR': pcr,
                    'PSR': psr,
                    'PBR': pbr,
                    '영업이익률': operating_margin,
                    'ROA': roa,
                    '매출액': revenue,
                    '영업이익': operating_income,
                    '당기순이익': net_income,
                    '영업활동으로인한현금흐름': cash_flow_from_operations
                })
            except KeyError as e:
                print(f"KeyError: {e} for stock code: {stock_code}")
        else:
            print(f"No data for stock code: {stock_code}")
    except Exception as e:
        print(f"No data for stock code: {stock_code} due to error: {e}")

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)

print("결과 데이터프레임:")
print(results_df)


종목코드 리스트:
['011200', '016380', '010060', '078930', '004000', '069260', '093370', '300720', '009150', '097950', '028260', '011170', '001740', '017670', '003090', '000120', '298050', '052690', '003670', '026960', '033780', '383220', '185750', '034730', '271560', '066570', '294870', '014680', '030000', '005300', '004490', '010130', '298020', '373220', '377300', '352820', '017800', '001040', '003850', '051600', '012750', '064350', '112610', '271940', '259960', '014820', '079550', '035720', '402340', '381970', '178920', '004370', '010120', '282330', '012450', '004990', '000210', '204320', '000880', '036460', '001230', '001120', '139480', '006260', '000990', '011210', '280360', '000080', '006400', '013890', '018880', '036570', '003410', '006280', '000660', '051910', '005250', '009830', '000670', '111770', '267250', '003240', '375500', '137310', '047040', '180640', '008930', '009420', '035420', '004020', '011780', '000270', '010780', '028670', '010950', '047050', '003490', '192820', '047810',

C:\Users\rkddk\AppData\Local\Temp\ipykernel_31688\188606302.py:45: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  fr_tables = pd.read_html(fr_page.text)
C:\Users\rkddk\AppData\Local\Temp\ipykernel_31688\188606302.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  fs_tables = pd.read_html(fs_page.text)
C:\Users\rkddk\AppData\Local\Temp\ipykernel_31688\188606302.py:45: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  fr_tables = pd.read_html(fr_page.text)
C:\Users\rkddk\AppData\Local\Temp\ipykernel_31688\188606302.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future versi

결과 데이터프레임:
       종목코드       날짜    PER    PCR   PSR   PBR 영업이익률   ROA       매출액     영업이익  \
0    011200  2022/12   0.95   0.87  0.51  0.46  53.5  46.1  185828.0  99494.0   
1    016380  2022/12   1.38   1.28  0.19  0.44   8.9  18.4   38197.0   3404.0   
2    010060  2022/12   1.74   1.49  0.55  0.39  27.2  14.1   27689.0   7524.0   
3    078930  2022/12   1.94   1.94  0.15  0.33  17.9   7.6  285825.0  51202.0   
4    004000  2022/12  10.01   5.42  0.59  0.64  16.4   5.4   24638.0   4043.0   
..      ...      ...    ...    ...   ...   ...   ...   ...       ...      ...   
148  020560  2022/12   9.43   0.95  0.17  1.51   9.7   0.2   62067.0   5988.0   
149  001680  2022/12   9.50   3.75  0.19  0.61   3.4   2.7   40841.0   1400.0   
150  069960  2022/12   9.58   2.73  0.28  0.29   6.4   1.8   50141.0   3209.0   
151  000720  2022/12   9.59   6.64  0.18  0.51   2.7   2.3  212391.0   5749.0   
152  004800  2022/12  90.36  11.66  0.38  0.54   1.7   0.5   37193.0    649.0   

        당기순이익  영

C:\Users\rkddk\AppData\Local\Temp\ipykernel_31688\188606302.py:61: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  fs_tables = pd.read_html(fs_page.text)


In [7]:
# 데이터프레임을 특정 디렉토리에 엑셀 파일로 저장
file_path = 'C:\\Users\\rkddk\\Downloads\\financedata.xlsx'
results_df.to_excel(file_path, index=False)

print(f"데이터프레임이 {file_path}에 저장되었습니다.")

데이터프레임이 C:\Users\rkddk\Downloads\financedata.xlsx에 저장되었습니다.
